# Transformer Classifier

This notebook trains a Transformer based classifier to predict inhaling and exhaling from video frames.

In [ ]:
from datetime import datetime
from pytz import timezone
import respiration.utils as utils

# The timestamp is the unique identifier for this training run
zone = timezone('Europe/Berlin')
model_id = datetime.now().astimezone(zone).strftime('%Y%m%d_%H%M%S')
device = utils.get_torch_device()

image_size = 224

# The manifest will store all the metadata for this training run
manifest = {
    'id': model_id,
    'device': str(device),
    'timestamp_start': datetime.now().astimezone().isoformat(),
    'dataset': 'VitalCamSet',
    'image_size': image_size,
}
model_id

## Dataset and Dataloader

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()
scenarios_all = dataset.get_scenarios(['101_natural_lighting'])

split_ratio = 0.8
manifest['split_ratio'] = split_ratio

training = scenarios_all[:int(len(scenarios_all) * split_ratio)]
manifest['training_scenarios'] = training

testing = scenarios_all[int(len(scenarios_all) * split_ratio):]
manifest['testing_scenarios'] = testing

In [ ]:
num_frames = 10
manifest['num_frames'] = num_frames

In [ ]:
import torch


def temporal_shifting_frames(frames: torch.Tensor) -> torch.Tensor:
    # Use unfold to create sliding windows
    new_video = frames.unfold(0, num_frames, 1)

    # Permute to (N, C, NumFrames, H, W)
    new_video = new_video.permute(0, 1, 4, 2, 3)
    return new_video


def temporal_shifting_signal(time_series: torch.Tensor) -> torch.Tensor:
    # Shift the signal that no negative values are present
    time_series = time_series - torch.min(time_series)

    # Calculate the difference between the time series
    diff = time_series[1:] - time_series[:-1]

    # Make all >0 values 1 and all <0 values 0
    diff = torch.where(diff > 0, torch.tensor(1.0, device=device), torch.tensor(0.0, device=device))

    # Make to int
    diff = diff.to(torch.long)

    return diff

In [ ]:
import math
import torch
from torchvision import transforms

import respiration.utils as utils


class ScenarioLoader:
    """
    A data loader for the VitalCamSet dataset. This class loads the video frames and the ground truth signal for a
    specific scenario. The video frames are loaded in chunks of a specific size. The ground truth signal is down-sampled
    to match the video frames' dimensions.
    """
    subject: str
    setting: str
    frames_per_segment: int

    def __init__(self,
                 subject: str,
                 setting: str,
                 frames_per_segment: int = 200):
        self.subject = subject
        self.setting = setting
        self.frames_per_segment = frames_per_segment

        self.video_path = dataset.get_video_path(subject, setting)
        self.total_frames = utils.get_frame_count(self.video_path)

    def __len__(self) -> int:
        return math.ceil(self.total_frames / self.frames_per_segment)

    def __iter__(self):
        self.current_index = 0
        return self

    def __next__(self):
        if self.current_index >= self.__len__():
            raise StopIteration
        else:
            item = self.__getitem__(self.current_index)
            self.current_index += 1
            return item

    def __getitem__(self, index) -> (torch.Tensor, torch.Tensor):
        """
        Return the frames and the ground truth signal for the given index
        :param index: The index of the chunk
        :return: The frames and the ground truth signal
        """

        if index >= self.__len__():
            raise IndexError("Index out of range")

        start = index * self.frames_per_segment
        end = start + self.frames_per_segment
        size = min(self.frames_per_segment, self.total_frames - start)

        # Load the video frames
        frames, _ = utils.read_video_rgb(self.video_path, size, start)
        preprocess = transforms.Compose([
            transforms.ToPILImage(mode='RGB'),
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor()
        ])
        frames = torch.stack([preprocess(frame) for frame in frames], dim=0)
        frames = frames.to(device)

        # Get the ground truth signal for the scenario
        gt_waveform = dataset.get_breathing_signal(self.subject, self.setting)
        gt_waveform = torch.tensor(gt_waveform, dtype=torch.float32, device=device)
        gt_waveform = torch.nn.functional.normalize(gt_waveform, dim=0)
        gt_waveform = gt_waveform[start:end]

        return frames, gt_waveform

In [ ]:
def count_classes(xxx):
    xxx = torch.argmax(xxx, dim=1)
    count = np.unique(xxx.cpu().detach().numpy(), return_counts=True)
    return count

## Model Training

In [ ]:
from torch.utils.tensorboard import SummaryWriter

log_dir = utils.dir_path('outputs', 'logs', model_id, mkdir=True)
writer = SummaryWriter(log_dir=log_dir)

In [ ]:
from vit_pytorch.simple_vit_3d import SimpleViT

# 2 classes: inhale, exhale
num_classes = 2
manifest['num_classes'] = num_classes

image_patch_size = 32
manifest['image_patch_size'] = image_patch_size

frame_patch_size = 2
manifest['frame_patch_size'] = frame_patch_size

depth = 6
manifest['depth'] = depth

heads = 8
manifest['heads'] = heads

mlp_dim = 2048
manifest['mlp_dim'] = mlp_dim

embedding_dim = 1024
manifest['embedding_dim'] = embedding_dim

model = SimpleViT(
    image_size=image_size,
    frames=num_frames,
    num_classes=num_classes,
    image_patch_size=image_patch_size,
    frame_patch_size=frame_patch_size,
    dim=embedding_dim,
    depth=depth,
    heads=heads,
    mlp_dim=mlp_dim,
).to(device)
manifest['base_model'] = 'SimpleViT_3D'

In [ ]:
epochs = 30
manifest['epochs'] = epochs

learning_rate = 0.0001
manifest['learning_rate'] = learning_rate

loss_fn = torch.nn.CrossEntropyLoss()
manifest['loss_fn'] = 'CrossEntropyLoss'

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
manifest['optimizer'] = 'AdamW'

In [ ]:
import numpy as np


def train_one_epoch(epoch_index: int):
    epoch_loss = 0.0

    # Iterate over the training scenarios
    for (subject, setting) in training:
        loader = ScenarioLoader(subject, setting)

        scenario_accuracy = 0.0

        # Iterate over the hole scenario video in chunks
        for idy, (frames, gt_classes) in enumerate(loader):
            frames = temporal_shifting_frames(frames)

            gt_classes = temporal_shifting_signal(gt_classes)
            # Cut the gt_classes to match the frames
            gt_classes = gt_classes[:frames.shape[0]]

            # Make predictions for this chunk
            outputs = model(frames)
            predicted_classes = outputs.argmax(dim=1)

            # Compute the loss and its gradients
            loss = loss_fn(outputs, gt_classes)

            # Optimize the model
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Gather data and report
            accuracy = (predicted_classes == gt_classes).float().mean()
            print(f'  {subject} #{idy:02d} outputs.count={count_classes(outputs)} accuracy={accuracy}')
            scenario_accuracy += accuracy

        scenario_accuracy /= len(loader)
        epoch_loss += scenario_accuracy

        print(f'  >> {subject} accuracy={scenario_accuracy}')
        writer.add_scalars('Training_Accuracy', {
            f'{subject}_{setting}': scenario_accuracy,
        }, epoch_index)
        writer.flush()

    return epoch_loss / len(training)

In [ ]:
model_dir = utils.dir_path('models', 'transformer', model_id, mkdir=True)

In [ ]:
def save_manifest(best_accuracy):
    manifest['trained_models'] = models
    manifest['best_testing_accuracy'] = float(best_accuracy)
    manifest['timestamp_finish'] = datetime.now().astimezone().isoformat()
    utils.write_json(os.path.join(model_dir, 'manifest.json'), manifest)

In [ ]:
import os
from tqdm.auto import tqdm

best_accuracy = 0.0
models = []

for epoch in tqdm(range(epochs)):
    print(f'Epoch {epoch}:')

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch)

    running_accuracy = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for inx, (subject, setting) in enumerate(testing):
            loader = ScenarioLoader(subject, setting)
            testing_accuracy = 0.0

            for (frames, gt_classes) in loader:
                frames = temporal_shifting_frames(frames)
                gt_classes = temporal_shifting_signal(gt_classes)
                gt_classes = gt_classes[:frames.shape[0]]
                voutputs = model(frames)
                testing_accuracy += (voutputs.argmax(dim=1) == gt_classes).float().mean()

            testing_accuracy /= len(loader)
            writer.add_scalars('Testing_Loss', {f'{subject}_{setting}': testing_accuracy}, epoch)
            print(f'  >> {subject} accuracy={testing_accuracy}')

            running_accuracy += testing_accuracy

    testing_accuracy = running_accuracy / len(testing)
    print(f'LOSS training={avg_loss} testing={testing_accuracy}')
    writer.add_scalars('Average_Accuracy', {
        'Training': avg_loss,
        'Testing': testing_accuracy,
    }, epoch)
    writer.flush()

    # Track the best performance, and save the model's state
    if testing_accuracy > best_accuracy:
        best_accuracy = testing_accuracy
        model_name = f'{model_id}_{epoch}.pth'

        model_path = os.path.join(model_dir, model_name)
        torch.save(model.state_dict(), model_path)

        models.append({
            'model': model_name,
            'epoch': epoch,
            'validation_accuracy': float(testing_accuracy),
        })
        save_manifest(testing_accuracy)

In [ ]:
save_manifest(best_accuracy)